In [12]:
from __future__ import annotations

from pathlib import Path
from ase.io import read

pdb_folder= "/home/suncc/Code/pub/himatcal/examples/EleCal/MD/pdb"
itp_folder="/home/suncc/Code/pub/himatcal/examples/EleCal/MD/itp"

mol_list = ["EC", "EMC", "DMC", "LiPF6"]


mass_fraction = {
    "EC": "24.69%",
    "EMC": "24.69%",
    "DMC": "49.38%",
    "LiPF6": "1.25%",
}

read_mol = {
    mol: read(f"{pdb_folder}/{mol}.pdb") for mol in mol_list
}

read_mol

FileNotFoundError: [Errno 2] No such file or directory: '/home/suncc/Code/pub/himatcal/examples/EleCal/MD/pdb/LiPF6.pdb'

In [ ]:


molar_mass = {
    "EC": 88.06,
    "EMC": 104.104,
    "DMC": 90.08,
    "LiPF6": 151.91,
}

molar_natoms = {
    "EC": 10,
    "EMC": 15,
    "DMC": 12,
    "LiPF6": 8,
}

total_natoms = 20000
density = 1.2


def calculate_compound(
    mass_fraction: dict[str, str],
    molar_mass: dict[str, float],
    molar_natoms: dict[str, int],
    total_natoms: int,
) -> dict[str, dict[str, int]]:
    def mass_to_molar(mass: float, substance: str) -> float:
        return mass / molar_mass[substance]

    def molar_to_mass(molar: float, substance: str) -> float:
        return molar * molar_mass[substance]

    def mass_fraction_to_molar_fraction(
        mass_fraction: dict[str, str],
    ) -> dict[str, float]:
        total_mass = sum(
            float(mass_fraction[component][:-1]) for component in mass_fraction
        )
        return {
            component: float(mass_fraction[component][:-1])
            / molar_mass[component]
            / total_mass
            for component in mass_fraction
        }

    def molar_fraction_to_mass_fraction(
        molar_fraction: dict[str, float],
    ) -> dict[str, str]:
        total_mass = sum(
            molar_fraction[component] * molar_mass[component]
            for component in molar_fraction
        )
        return {
            component: f"{(molar_fraction[component] * molar_mass[component] / total_mass) * 100:.2f}%"
            for component in molar_fraction
        }

    molar_fraction = mass_fraction_to_molar_fraction(mass_fraction)
    min_molar_fraction = min(molar_fraction.values())
    molar_fraction = {
        component: molar_fraction[component] / min_molar_fraction
        for component in molar_fraction
    }
    atoms_fraction = {
        component: molar_fraction[component] * molar_natoms[component]
        for component in molar_fraction
    }
    tot_atom = sum(atoms_fraction.values())
    ratio = total_natoms / tot_atom
    round_compound = {
        component: round(molar_fraction[component] * ratio)
        for component in molar_fraction
    }
    compound = {
        component: {
            "mol": round_compound[component],
            "n_atoms": round_compound[component] * molar_natoms[component],
        }
        for component in round_compound
    }
    return compound


compound = calculate_compound(mass_fraction, molar_mass, molar_natoms, total_natoms)


def extracted_LiPF6(componds_dict, LiPF6_name):
    MOL_LPF6 = componds_dict[LiPF6_name]["mol"]
    del componds_dict[LiPF6_name]
    componds_dict["PF6"] = {"mol": MOL_LPF6, "n_atoms": MOL_LPF6 * 7}
    componds_dict["Li"] = {"mol": MOL_LPF6, "n_atoms": MOL_LPF6 * 1}
    return componds_dict


compound = extracted_LiPF6(compound, "LiPF6")
print(compound)

# write the compound to a file
with Path.open(Path("compound.json"), "w") as f:
    f.write(str(compound))


In [ ]:
from himatcal.recipes.electrolyte.sol_stru.build_box import ElectrolyteBuilder

builder = ElectrolyteBuilder(
    mol_path="/home/suncc/Code/pub/himatcal/examples/EleCal/MD/pdb"
)
builder.build_box(compound, density=density, box=None, save_path="./box.pdb")


/home/suncc/Code/Archived/mbuild/mbuild/compound.py:997: UserWarning: The Compound you are adding has a box. The box of the parent compound will be used. Use inherit_box = True if you wish to replace the parent compound box with that of Compound being added.
  warn(


In [21]:
mass_fraction_converted = molar_fraction_to_mass_fraction(molar_fraction)
print(mass_fraction_converted)

{'EC': '22.19%', 'EMC': '22.19%', 'DMC': '44.38%', 'LiPF6': '11.25%'}
